Here we use Hartley's method.

In [1]:
import numpy as np
import cv2

We load the previously recorded images, and get the chessboard corners positions with a subpixel precision.

In [2]:
lefts=[]
leftsGray=[]
rights=[]
rightsGray=[]

for i in np.arange(0,60):
    l=cv2.imread("left_"+str(i)+".png")
    r=cv2.imread("right_"+str(i)+".png")
    lefts.append(l)
    rights.append(r)
    leftsGray.append(cv2.cvtColor(l,cv2.COLOR_BGR2GRAY))
    rightsGray.append(cv2.cvtColor(r,cv2.COLOR_BGR2GRAY))

In [3]:
def getAllChessboardCorners(images):
    res=[]
    for image in images:
        found,corners=cv2.findChessboardCorners(image,(9,6))
        if not found:
            print("Corners not found!")
        cv2.cornerSubPix(image, corners, (11,11), (-1,-1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))
        res.append(corners)
    return res

In [4]:
pattern=np.zeros((9*6,3),np.float32)
pattern[:,:2]=np.mgrid[0:9,0:6].T.reshape(-1,2)

h,w=leftsGray[0].shape

In [5]:
leftCorners=getAllChessboardCorners(leftsGray)
rightCorners=getAllChessboardCorners(rightsGray)
patterns=len(leftCorners)*[pattern]

In [6]:
leftCalibWorked,leftMtx,leftDist,_,_=cv2.calibrateCamera(patterns,leftCorners,(w,h),None,None)
rightCalibWorked,rightMtx,rightDist,_,_=cv2.calibrateCamera(patterns,rightCorners,(w,h),None,None)

Not used here, but could be useful: adjust the scale factor to get minimum pixels outside of the original image.

In [7]:
optimalLeftMatrix,optimalLeftROI=cv2.getOptimalNewCameraMatrix(leftMtx,leftDist,(w,h),1)
optimalRightMatrix,optimalRightROI=cv2.getOptimalNewCameraMatrix(rightMtx,rightDist,(w,h),1)

In [8]:
if not leftCalibWorked or not rightCalibWorked:
    print("Calibration didn't work")
    print("\t right:"+str(leftCalibWorked))
    print("\t left:"+str(rightCalibWorked))
else:
    print("Optimal left: \n"+str(optimalLeftMatrix))
    print("leftmatrix: \n"+str(leftMtx))
    print("Optimal right: \n"+str(optimalRightMatrix))
    print("rightmatrix: \n"+str(rightMtx))

Optimal left: 
[[ 175.65402222    0.          123.09808778]
 [   0.          135.91906738  159.31235208]
 [   0.            0.            1.        ]]
leftmatrix: 
[[ 190.44984831    0.          130.19296664]
 [   0.          145.81503843  148.39940135]
 [   0.            0.            1.        ]]
Optimal right: 
[[ 172.47463989    0.          115.31757619]
 [   0.          129.77236938  148.21178242]
 [   0.            0.            1.        ]]
rightmatrix: 
[[ 183.87004286    0.          122.34454987]
 [   0.          138.26688487  145.18068126]
 [   0.            0.            1.        ]]


In [9]:
_,_,_,_,_,R,T,E,F=cv2.stereoCalibrate(patterns,leftCorners,rightCorners,optimalLeftMatrix,leftDist,optimalRightMatrix,rightDist,(w,h))

In [10]:
print("R: \n"+str(R))
print("T: \n"+str(T))
print("E: \n"+str(E))
print("F: \n"+str(F))

R: 
[[ 0.99961086 -0.01329827  0.0245212 ]
 [ 0.01181341  0.99814435  0.05973524]
 [-0.02527007 -0.05942232  0.99791303]]
T: 
[[-1.61193622]
 [ 0.15110279]
 [-0.34053764]]
E: 
[[  2.04531646e-04   3.30926843e-01   1.71129540e-01]
 [ -3.81138862e-01  -9.12564293e-02   1.60022177e+00]
 [ -1.70086445e-01  -1.60693563e+00  -9.99946247e-02]]
F: 
[[ -3.26008927e-08  -6.81677074e-05   6.07268529e-03]
 [  8.07412046e-05   2.49834941e-05  -7.34649478e-02]
 [ -7.28715221e-03   6.12494506e-02   1.00000000e+00]]


In [11]:
leftCornersArray=np.concatenate(leftCorners)
rightCornersArray=np.concatenate(rightCorners)

undistortedLeft=cv2.undistortPoints(leftCornersArray,leftMtx,leftDist,P=optimalLeftMatrix)
undistortedRight=cv2.undistortPoints(rightCornersArray,rightMtx,rightDist,P=optimalRightMatrix)

In [12]:
_,H1,H2=cv2.stereoRectifyUncalibrated(undistortedLeft,undistortedRight,F,(w,h))

In [13]:
RLeft=np.linalg.inv(optimalLeftMatrix).dot(H1).dot(optimalLeftMatrix)
RRight=np.linalg.inv(optimalRightMatrix).dot(H2).dot(optimalRightMatrix)
leftMx,leftMy=cv2.initUndistortRectifyMap(leftMtx,leftDist,RLeft,optimalLeftMatrix,(w,h),cv2.CV_32F)
rightMx,rightMy=cv2.initUndistortRectifyMap(rightMtx,rightDist,RRight,optimalRightMatrix,(w,h),cv2.CV_32F)

In [14]:
resLeft=cv2.remap(leftsGray[1],leftMx,leftMy,cv2.INTER_LINEAR)
resRight=cv2.remap(rightsGray[1],rightMx,rightMy,cv2.INTER_LINEAR)

In [15]:
cv2.imshow("left remap",resLeft)
cv2.imshow("right remap",resRight)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
b_matcher=cv2.StereoBM_create(numDisparities=16,blockSize=15)
stereo=b_matcher.compute(resLeft,resRight).astype(np.float32)
r=cv2.normalize(stereo,r,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U)

In [18]:
cv2.imshow("disparity",r)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
from IPython.html.widgets import interact, interactive
from IPython.display import clear_output, display

def compute_disparity(nDisparities,blksize,imNum):
    b_matcher=cv2.StereoBM_create(numDisparities=nDisparities,blockSize=blksize)
    resLeft=cv2.remap(leftsGray[imNum],leftMx,leftMy,cv2.INTER_LINEAR)
    resRight=cv2.remap(rightsGray[imNum],rightMx,rightMy,cv2.INTER_LINEAR)
    cv2.imshow("Diparity map",cv2.normalize(b_matcher.compute(resLeft,resRight).astype(np.float32),None,alpha=0,beta=255,norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8U))
    cv2.waitKey(10)
    
w=interactive(compute_disparity,nDisparities=(32,64,32),blksize=(5,91,2),imNum=(0,len(leftsGray)))
clear_output(wait=True)
display(w)

Don't forget to save your rectification mapping!

In [ ]:
np.savetxt('leftMx',leftMx)
np.savetxt('leftMy',leftMy)

np.savetxt('rightMx',rightMx)
np.savetxt('rightMy',rightMy)